In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sl

In [2]:
# соединение с базой sqlite
con = sl.connect('my-test.db')

In [3]:
members_13 = pd.read_csv('members_13.csv', parse_dates=['birthday','entry_date'])
members_13

,member_name,birthday,sex,profession,salary_usd$,entry_date,group_id,city_name,state_name,age,fun_club_name,category_id,monthly_fee_USD$,category_name,membership_duration_days,"sum_fee_paid, USD$"
0,Vanessa Casey,1960-08-08,F,"Engineer, manufacturing systems",1884,2018-05-24,19927387,Montgomery,West Virginia,63,Events That Make You Smarter,6,7,Education & Learning,1803,421.0
1,Cheryl Carey,1958-12-11,N,Trade union research officer,1518,2019-11-14,19712846,Colonia,New Jersey,65,Mix Professionals Chicago,9,18,Fitness,1264,758.0
2,Kendra Valdez,1962-06-16,M,Press photographer,1669,2018-08-06,10528562,Pinckney,Michigan,61,Madison Square Park Mommies!,25,19,Parents & Family,1729,1095.0
3,Toni Baxter,1992-03-21,F,Emergency planning/management officer,3818,2017-07-18,1291778,Florence,Kentucky,31,San Francisco Dodgeball Meetup Group,32,1,Sports & Recreation,2113,70.0
4,Shaun Zuniga,1999-07-04,N,"Runner, broadcasting/film/video",4962,2017-11-18,26053931,Nickerson,Kansas,24,Passionate Concert Goers,21,9,Music,1990,597.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130984,George Mcdaniel,1943-03-14,M,Comptroller,2393,2018-04-14,1222759,Oakhurst,California,80,RHYTHMflow Explorations,21,21,Music,1843,1290.0
130985,Kevin Diaz DVM,1999-05-16,F,Customer service manager,4874,2018-12-27,2137821,Spring Lake Park,Minnesota,24,Aquatic Voyagers Scuba Club,23,18,Outdoors & Adventure,1586,952.0
130986,Latasha Alvarez MD,1953-07-07,N,Interpreter,2579,2018-09-20,23343110,Newton County,Missouri,70,Chicago Womens Soccer Meet up,32,5,Sports & Recreation,1684,281.0
130987,Hector Davidson,1945-05-05,N,Lexicographer,2890,2018-11-18,10982362,Houston,Alaska,78,Chicago Indonesian (Bahasa Indonesia) Language...,16,23,Language & Ethnic Identity,1625,1246.0


In [4]:
members_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130989 entries, 0 to 130988
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   member_name               130989 non-null  object        
 1   birthday                  130989 non-null  datetime64[ns]
 2   sex                       130989 non-null  object        
 3   profession                130989 non-null  object        
 4   salary_usd$               130989 non-null  int64         
 5   entry_date                130989 non-null  datetime64[ns]
 6   group_id                  130989 non-null  int64         
 7   city_name                 130989 non-null  object        
 8   state_name                130989 non-null  object        
 9   age                       130989 non-null  int64         
 10  fun_club_name             130974 non-null  object        
 11  category_id               130989 non-null  int64         
 12  mo

In [6]:
# загружаем датасет в базу sqllite
members_13.to_sql('members_13', con)

130989

In [5]:
pd.read_sql('''SELECT* FROM members_13 LIMIT 3''', con, index_col='index')

,member_name,birthday,sex,profession,salary_usd$,entry_date,group_id,city_name,state_name,age,fun_club_name,category_id,monthly_fee_USD$,category_name,membership_duration_days,"sum_fee_paid, USD$"
index,,,,,,,,,,,,,,,,
0,Vanessa Casey,1960-08-08 00:00:00,F,"Engineer, manufacturing systems",1884,2018-05-24 00:00:00,19927387,Montgomery,West Virginia,63,Events That Make You Smarter,6,7,Education & Learning,1803,421.0
1,Cheryl Carey,1958-12-11 00:00:00,N,Trade union research officer,1518,2019-11-14 00:00:00,19712846,Colonia,New Jersey,65,Mix Professionals Chicago,9,18,Fitness,1264,758.0
2,Kendra Valdez,1962-06-16 00:00:00,M,Press photographer,1669,2018-08-06 00:00:00,10528562,Pinckney,Michigan,61,Madison Square Park Mommies!,25,19,Parents & Family,1729,1095.0


In [23]:
# динамика количества новых членов клубов в разрезе категорий интересов с 2017 по 2020 гг.
pd.read_sql('''SELECT category_name,
            COUNT(CASE WHEN strftime('%Y', entry_date)='2017' THEN member_name END) as count_members_2017,
            COUNT(CASE WHEN strftime('%Y', entry_date)='2018' THEN member_name END) as count_members_2018,
            COUNT(CASE WHEN strftime('%Y', entry_date)='2019' THEN member_name END) as count_members_2019,
            COUNT(CASE WHEN strftime('%Y', entry_date)='2020' THEN member_name END) as count_members_2020
            FROM members_13 GROUP BY category_name''', con)

,category_name,count_members_2017,count_members_2018,count_members_2019,count_members_2020
0,Arts & Culture,1394,1975,2047,676
1,Book Clubs,624,887,957,323
2,Cars & Motorcycles,232,344,321,122
3,Community & Environment,1337,1893,1991,625
4,Dancing,753,1162,1112,348
5,Education & Learning,1545,2107,2197,685
6,Fitness,1227,1750,1690,581
7,Food & Drink,1891,2726,2695,865
8,Games,1053,1499,1587,526
9,Health & Wellbeing,3265,4656,4727,1526


In [18]:
# динамика количества новых членов клубов в разрезе категорий интересов с 2017 по 2020 гг.
# или то же через FILTER с разбивкой на 3 корзины значений по TOTAL
pd.read_sql('''SELECT category_name,count_members_2017,count_members_2018,count_members_2019,count_members_2020,
            TOTAL, NTILE(3) OVER (ORDER BY TOTAL) as RANK FROM
            (SELECT category_name, 
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2017') AS count_members_2017,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2018') AS count_members_2018,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2019') AS count_members_2019,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2020') AS count_members_2020,
            COUNT(member_name) as TOTAL
            FROM members_13 GROUP BY category_name ORDER BY TOTAL DESC) as query
            ORDER BY TOTAL DESC''', con)

,category_name,count_members_2017,count_members_2018,count_members_2019,count_members_2020,TOTAL,RANK
0,Health & Wellbeing,3265,4656,4727,1526,14174,3
1,Language & Ethnic Identity,2430,3635,3521,1202,10788,3
2,New Age & Spirituality,2093,2947,3051,994,9085,3
3,Outdoors & Adventure,2025,2879,2934,892,8730,3
4,Sports & Recreation,1857,2766,2768,886,8277,3
5,Food & Drink,1891,2726,2695,865,8177,3
6,Education & Learning,1545,2107,2197,685,6534,3
7,Music,1378,2085,2075,688,6226,3
8,Arts & Culture,1394,1975,2047,676,6092,2
9,Community & Environment,1337,1893,1991,625,5846,2


In [28]:
# РАСШИРЕНИЕ: динамика количества новых членов клубов в разрезе категорий интересов и пола участников с 2017 по 2020 гг.
# или то же через FILTER с разбивкой на 3 корзины значений по TOTAL
pd.read_sql('''SELECT sex, category_name,count_members_2017,count_members_2018,count_members_2019,count_members_2020,
            TOTAL, NTILE(3) OVER (ORDER BY TOTAL) as RANK FROM
            (SELECT sex, category_name, 
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2017') AS count_members_2017,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2018') AS count_members_2018,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2019') AS count_members_2019,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2020') AS count_members_2020,
            COUNT(member_name) as TOTAL
            FROM members_13 GROUP BY sex, category_name ORDER BY TOTAL DESC) as query
            ORDER BY TOTAL DESC''', con).head(20)

,sex,category_name,count_members_2017,count_members_2018,count_members_2019,count_members_2020,TOTAL,RANK
0,N,Health & Wellbeing,1108,1577,1587,497,4769,3
1,F,Health & Wellbeing,1087,1534,1664,472,4757,3
2,M,Health & Wellbeing,1070,1545,1476,557,4648,3
3,M,Language & Ethnic Identity,815,1247,1227,402,3691,3
4,F,Language & Ethnic Identity,776,1216,1179,393,3564,3
5,N,Language & Ethnic Identity,839,1172,1115,407,3533,3
6,N,New Age & Spirituality,701,1003,1034,363,3101,3
7,F,New Age & Spirituality,686,1000,1052,330,3068,3
8,N,Outdoors & Adventure,680,1027,1012,304,3023,3
9,M,New Age & Spirituality,706,944,965,301,2916,3


In [19]:
# динамика сумм уплаченных членских взносов в разрезе категорий интересов с 2017 по 2020 гг.
# с разбивкой на 3 корзины значений по TOTAL
pd.read_sql('''SELECT category_name,sum_fees_2017,sum_fees_2018,sum_fees_2019,sum_fees_2020,
            TOTAL, NTILE(3) OVER (ORDER BY TOTAL) as RANK FROM
            (SELECT category_name, 
            SUM (monthly_fee_USD$) FILTER (WHERE strftime('%Y', entry_date)='2017') AS sum_fees_2017,
            SUM (monthly_fee_USD$) FILTER (WHERE strftime('%Y', entry_date)='2018') AS sum_fees_2018,
            SUM (monthly_fee_USD$) FILTER (WHERE strftime('%Y', entry_date)='2019') AS sum_fees_2019,
            SUM (monthly_fee_USD$) FILTER (WHERE strftime('%Y', entry_date)='2020') AS sum_fees_2020,
            SUM (monthly_fee_USD$) as TOTAL
            FROM members_13 GROUP BY category_name ORDER BY TOTAL DESC) as query
            ORDER BY TOTAL DESC''', con)

,category_name,sum_fees_2017,sum_fees_2018,sum_fees_2019,sum_fees_2020,TOTAL,RANK
0,Health & Wellbeing,39035,56550,57117,18692,171394,3
1,Language & Ethnic Identity,29641,43594,42359,14351,129945,3
2,Outdoors & Adventure,25720,36496,36334,11425,109975,3
3,New Age & Spirituality,24897,34976,35718,11889,107480,3
4,Food & Drink,22690,32708,33015,10492,98905,3
5,Sports & Recreation,21934,32589,33414,10649,98586,3
6,Education & Learning,18364,24929,25950,8260,77503,3
7,Arts & Culture,17225,23700,24426,8221,73572,3
8,Music,16343,23876,23691,7891,71801,2
9,Community & Environment,16187,22881,24356,7646,71070,2


In [20]:
# динамика количества новых членов клубов в разрезе штатов с 2017 по 2020 гг.
# или то же через FILTER с разбивкой на 3 корзины значений по TOTAL
pd.read_sql('''SELECT state_name,count_members_2017,count_members_2018,count_members_2019,count_members_2020,
            TOTAL, NTILE(3) OVER (ORDER BY TOTAL) as RANK FROM
            (SELECT state_name, 
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2017') AS count_members_2017,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2018') AS count_members_2018,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2019') AS count_members_2019,
            COUNT(member_name) FILTER (WHERE strftime('%Y', entry_date)='2020') AS count_members_2020,
            COUNT(member_name) as TOTAL
            FROM members_13 GROUP BY state_name ORDER BY TOTAL DESC) as query
            ORDER BY TOTAL DESC''', con)

,state_name,count_members_2017,count_members_2018,count_members_2019,count_members_2020,TOTAL,RANK
0,Texas,1943,2849,2815,932,8539,3
1,California,1684,2399,2475,726,7284,3
2,New York,1595,2317,2208,722,6842,3
3,Pennsylvania,1457,2136,2272,715,6580,3
4,Florida,1298,1972,1943,643,5856,3
5,Illinois,1291,1872,1875,596,5634,3
6,Ohio,1166,1689,1685,575,5115,3
7,North Carolina,874,1227,1232,440,3773,3
8,Missouri,799,1222,1193,405,3619,3
9,Michigan,830,1127,1163,371,3491,3
